In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
pip install keras-tuner

In [3]:
import keras_tuner as kt

In [4]:
(train_img, train_labels), (test_img, test_labels) = keras.datasets.fashion_mnist.load_data()

In [5]:
train_img = train_img.astype('float32')/255.0
test_img = test_img.astype('float32')/255.0

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     project_name='intro to kt',
                     directory='my_dir')

In [9]:
early_stopping =tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [11]:
tuner.search(train_img,train_labels,validation_split=0.2,callbacks=[early_stopping],epochs=50)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 27s]
val_accuracy: 0.8598333597183228

Best val_accuracy So Far: 0.8926666378974915
Total elapsed time: 00h 15m 24s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [12]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_img,train_labels,epochs=50,validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']

best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4986 - accuracy: 0.8229 - val_loss: 0.3966 - val_accuracy: 0.8597
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3695 - accuracy: 0.8665 - val_loss: 0.3514 - val_accuracy: 0.8720
Epoch 3/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3302 - accuracy: 0.8790 - val_loss: 0.3517 - val_accuracy: 0.8758
Epoch 4/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3066 - accuracy: 0.8869 - val_loss: 0.3381 - val_accuracy: 0.8761
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2881 - accuracy: 0.8924 - val_loss: 0.3293 - val_accuracy: 0.8799
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2725 - accuracy: 0.8985 - val_loss: 0.3260 - val_accuracy: 0.8854
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2584 - accuracy: 0.9047 - val_loss: 0.3549 - val_accuracy:

In [15]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_img, train_labels, epochs=best_epoch, validation_split=0.2)

Epoch 1/31
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4918 - accuracy: 0.8258 - val_loss: 0.4071 - val_accuracy: 0.8574
Epoch 2/31
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3711 - accuracy: 0.8653 - val_loss: 0.3735 - val_accuracy: 0.8664
Epoch 3/31
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3329 - accuracy: 0.8767 - val_loss: 0.3335 - val_accuracy: 0.8805
Epoch 4/31
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3061 - accuracy: 0.8881 - val_loss: 0.3425 - val_accuracy: 0.8737
Epoch 5/31
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2873 - accuracy: 0.8939 - val_loss: 0.3454 - val_accuracy: 0.8763
Epoch 6/31
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2699 - accuracy: 0.8989 - val_loss: 0.3477 - val_accuracy: 0.8745
Epoch 7/31
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2565 - accuracy: 0.9039 - val_loss: 0.3232 - val_accuracy:

In [17]:
eval_result = hypermodel.evaluate(test_img,test_labels)
print("[test loss, test accuracy]:", eval_result)



313/313 [==============================] - 1s 3ms/step - loss: 0.4569 - accuracy: 0.8910
[test loss, test accuracy]: [0.4568565785884857, 0.890999972820282]
